<a href="https://colab.research.google.com/github/yasminbv/DataScience/blob/main/Homework6_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Data description is provided below
import pandas as pd
import numpy as np
df = pd.read_csv('https://raw.githubusercontent.com/ogut77/DataScience/main/data/Leads.csv')
df.info()

In [ ]:
 #There are missing values in the Lead data .
# Replace missing variables with the mean  if variable is numeric or mode if variable is categorical.
for col in df.columns:
    if ((df[col].dtype == 'float64') or (df[col].dtype == 'float64')) :
        df[col] = df[col].fillna(df[col].mean())
    else:
        df[col] = df[col].fillna(df[col].mode()[0])


In [ ]:
#Check that there are no missing data
df.isnull().sum()

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

le = LabelEncoder()
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = le.fit_transform(df[col])

df = df.drop(columns=['Prospect ID','Lead Number'])
y = df['Converted']
X = df.drop(columns=['Converted'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

In [ ]:
def eval(yact,ypred):
 from sklearn import metrics
 print("Accuracy:",metrics.accuracy_score(yact, ypred))
 print("Precision:",metrics.precision_score(yact, ypred))
 print("Recall:",metrics.recall_score(yact, ypred))
 
 cnf_matrix = metrics.confusion_matrix(yact, ypred)
 import numpy as np
 import matplotlib.pyplot as plt
 import seaborn as sns
 %matplotlib inline
 class_names=[0,1] # name  of classes
 fig, ax = plt.subplots()
 tick_marks = np.arange(len(class_names))
 plt.xticks(tick_marks, class_names)
 plt.yticks(tick_marks, class_names)
 # create heatmap
 sns.heatmap(pd.DataFrame(cnf_matrix), annot=True, cmap="YlGnBu" ,fmt='g')
 ax.xaxis.set_label_position("top")
 plt.tight_layout()
 plt.title('Confusion matrix', y=1.1)
 plt.ylabel('Actual label')
 plt.xlabel('Predicted label')


In [ ]:
#1)With optuna method and using train data  choose best method among Random Forest","XGBoost", "LightGBM","GradientBoostingClassifier"
#and optimeze these method based on following parameters.
#max_depth = trial.suggest_int("max_depth", 2,X_train.shape[1])
#max_features = trial.suggest_int("max_features", 2,X_train.shape[1])
#Evaluate performance of the best method on test data using eval function defined above.
#Note: This should be similar to what we have done in the exercises in class. 
!pip install optuna

In [ ]:
import optuna
import sklearn.datasets
import sklearn.ensemble
import sklearn.model_selection
import sklearn.svm

def objective(trial):
    x, y = X_train,y_train

    regressor_name = trial.suggest_categorical("regressor", ["GradientBoosting", "XGBoost","LightGBM","RandomForest"])
    if regressor_name == "GradientBoosting":
         max_depth = trial.suggest_int("max_depth", 2,X_train.shape[1])
         max_features = trial.suggest_int("max_features", 2,X_train.shape[1])
         regressor_obj = sklearn.ensemble.GradientBoostingRegressor(random_state=17,  max_depth=max_depth, max_features=max_features )
    
    elif regressor_name == "XGBoost":
         from xgboost import XGBRegressor
         max_depth = trial.suggest_int("max_depth", 2,X_train.shape[1])
         max_features = trial.suggest_int("max_features", 2,X_train.shape[1])
         regressor_obj = XGBRegressor(random_state=17,  max_depth=max_depth, max_features=max_features )
        
    elif regressor_name == "LightGBM":
         import lightgbm as lgb
         max_depth = trial.suggest_int("max_depth", 2,X_train.shape[1])
         max_features = trial.suggest_int("max_features", 2,X_train.shape[1])
         regressor_obj = lgb.LGBMRegressor(random_state=17,  max_depth=max_depth, max_features=max_features )
        
    else:
         max_depth = trial.suggest_int("max_depth", 2,X_train.shape[1])
         max_features = trial.suggest_int("max_features", 2,X_train.shape[1])
         regressor_obj = sklearn.ensemble.RandomForestRegressor(random_state=17,  max_depth=max_depth, max_features=max_features )

    r2=sklearn.model_selection.cross_val_score(regressor_obj, x, y, n_jobs=-1, cv=3).mean()
   
    return r2

if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=20)
    print(study.best_trial)

In [ ]:
params={ 'max_features': 2,X_train.shape[1],'max_depth': 2,X_train.shape[1] }
import lightgbm as lgb
lgbm = lgb.LGBMClassifier(random_state=17,**params)
lgbm.fit(X_train, y_train)
y_predlgbm = lgbm.predict(X_test)
eval(y_test, y_predlgbm)

from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor(random_state=17)
rfr.fit(X_train,y_train)
y_predrfr = rfr.predict(X_test)
eval(y_test,y_predrfr)

from sklearn.ensemble import GradientBoostingRegressor
from time import time
gbr = GradientBoostingRegressor(random_state=17)
gbr.fit(X_train, y_train)
y_predgbr = gbr.predict(X_test)
eval(y_test,y_predgbr)

from xgboost import XGBRegressor
xgr = XGBRegressor(random_state=17)
xgr.fit(X_train, y_train)
y_predxgr = xgr.predict(X_test)
evalmetric(y_test,y_predxgr)

In [ ]:
#2)Using pycaret AutoML tools,make prediction on test data and evaluate performance of prediction using eval function defined above.
!pip install pycaret

In [ ]:
result = pd.concat([X_train, y_train], axis=1)
from pycaret.classification import *
s = setup(data = result, target = 'stroke', session_id=123,fold=5)
tuned2 = tune_model(best_model)
tuned_predictions = predict_model(tuned2, data=X_test)
eval(y_test,tuned_predictions['prediction_label'])

In [ ]:
#3) Using Autogluon  AutoML tools,make prediction on test data and evaluate performance of prediction using eval function defined above.
!pip install autogluon

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from autogluon.tabular import TabularDataset, TabularPredictor 
predictor= TabularPredictor(label ='stroke').fit(train_data =result, verbosity = 2)
y_pred = predictor.predict(X_test)
y_pred=pd.DataFrame(y_pred,columns=['stroke'])
eval(y_test,y_pred)